In [1]:
import numpy as np
import pandas as pd
from datasets import load_dataset

import torch
from transformers import AutoTokenizer, AutoModel, BertTokenizer, AutoModelForSequenceClassification, pipeline, trainer

tokenizer = AutoTokenizer.from_pretrained('ernie-1.0')
# model = AutoModelForSequenceClassification.from_pretrained('ernie-1.0')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
max_len = 200

# raw_datasets = load_dataset("imdb")

In [2]:
def tokenize_function(examples):
    return tokenizer(examples['text'], max_length=200, padding='max_length', return_tensors = 'pt')

# tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [3]:
chn_train = load_dataset('seamew/ChnSentiCorp')
# BATCH=32
# MAX_SEQLEN=300
# LR=5e-5
# EPOCH=10

Using custom data configuration default
Reusing dataset chn_senti_corp (C:\Users\ZK Hao\.cache\huggingface\datasets\seamew___chn_senti_corp\default\0.0.0\1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
def process_data(data):
    # max length is about 306..., truncation and padding are needed
    return tokenizer(data["text"], max_length=300, padding='max_length', truncation=True)


In [5]:
tokenized_datasets = chn_train.map(process_data)

Loading cached processed dataset at C:\Users\ZK Hao\.cache\huggingface\datasets\seamew___chn_senti_corp\default\0.0.0\1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85\cache-830c42192a366b04.arrow


  0%|          | 0/1200 [00:00<?, ?ex/s]

Loading cached processed dataset at C:\Users\ZK Hao\.cache\huggingface\datasets\seamew___chn_senti_corp\default\0.0.0\1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85\cache-5ec2220bc07689ab.arrow


In [6]:
# remove the select if want to train on the full set
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(96))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(12))
# test_set = tokenized_datasets["dev"].shuffle(seed=42)

Loading cached shuffled indices for dataset at C:\Users\ZK Hao\.cache\huggingface\datasets\seamew___chn_senti_corp\default\0.0.0\1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85\cache-dc7e3749cf690cab.arrow
Loading cached shuffled indices for dataset at C:\Users\ZK Hao\.cache\huggingface\datasets\seamew___chn_senti_corp\default\0.0.0\1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85\cache-b920ebcb914336a0.arrow


The following aprt is about danmu data prrocessing, having some problem...

In [ ]:
# this is for danmu processing? have some problem
danmuku = pd.read_csv('test_dataframe.csv')

In [35]:
# danmuku['text']
def process_csv(data):
    temp = tokenizer(data['text'], max_length=300, padding='max_length', truncation=True)
    return temp.attention_mask, temp.token_type_ids, temp.input_ids
tokenized_datasets = danmuku.apply(process_data, axis=1)

In [40]:
tokenized_datasets = tokenized_datasets.apply(lambda x: pd.Series([x.attention_mask, x.token_type_ids, x.input_ids], index=['attention_mask', 'token_type_ids', 'input_ids']))

In [44]:
small_train_dataset_danmu = tokenized_datasets[: 800]
small_eval_dataset_danmu = tokenized_datasets[800 :]

this is the first training part

In [7]:
from transformers import Trainer, TrainingArguments

# the original not finetuned model, num label depends on the data labels
model = AutoModelForSequenceClassification.from_pretrained('ernie-1.0', num_labels=2)

Some weights of the model checkpoint at ernie-1.0 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ernie-1.0 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRA

In [8]:
# set eval steps to larger (100) if train on the full set
training_args = TrainingArguments('finetune_trainer',
                                  evaluation_strategy="steps",
                                  logging_dir='logs/',
                                  warmup_steps=50,
                                  eval_steps=20,
                                  logging_steps=20,
                                  per_device_train_batch_size=4,
                                  per_device_eval_batch_size=4,
                                  save_total_limit = 3,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="accuracy" # check???
                                  )

In [9]:
from datasets import load_metric
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [10]:
torch.cuda.empty_cache()

In [11]:
model.to(device)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
model.save_pretrained("Ernie-finetuned")

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 96
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 72


Step,Training Loss,Validation Loss,Accuracy
20,0.709500,0.889223,0.500000
40,0.409500,0.716807,0.416667
60,0.224400,0.749353,0.750000


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 12
  Batch size = 4
Saving model checkpoint to finetune_trainer\checkpoint-20
Configuration saved in finetune_trainer\checkpoint-20\config.json
Model weights saved in finetune_trainer\checkpoint-20\pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 12
  Batch size = 4
Saving model checkpoint to finetune_trainer\checkpoint-40
Configuration saved in finetune_trainer\checkpoint-40\config.json
Model weights saved in finetune_trainer\checkpoint-40\pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running

In [12]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 12
  Batch size = 4


{'eval_loss': 0.7493529319763184,
 'eval_accuracy': 0.75,
 'eval_runtime': 0.9192,
 'eval_samples_per_second': 13.054,
 'eval_steps_per_second': 3.264,
 'epoch': 3.0}

In [15]:
test_set = tokenized_datasets["validation"].shuffle(seed=42).select(range(100))

Loading cached shuffled indices for dataset at C:\Users\ZK Hao\.cache\huggingface\datasets\seamew___chn_senti_corp\default\0.0.0\1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85\cache-05f0fcbf7c12bc18.arrow


In [57]:
# load from fine-tuned model
from transformers import Trainer, TrainingArguments

# batch_size depends on the GPU memory

training_args = TrainingArguments('finetune_trainer',
                                  evaluation_strategy='steps',
                                  eval_steps=100,
                                  num_train_epochs=5,
                                  per_device_train_batch_size=4,
                                  per_device_eval_batch_size=4,
                                  metric_for_best_model="accuracy" # check???
                                  )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [58]:
torch.cuda.empty_cache()
new_model = AutoModelForSequenceClassification.from_pretrained("finetune_trainer/checkpoint-500")
new_model.to(device)

loading configuration file finetune_trainer/checkpoint-500\config.json
Model config BertConfig {
  "_name_or_path": "ernie-1.0",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "relu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 513,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 18000
}

loading weights file finetune_trainer/checkpoint-500\pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initia

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(18000, 768, padding_idx=0)
      (position_embeddings): Embedding(513, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05, element

In [59]:
evaluator = Trainer(
    model=new_model,
    train_dataset=small_train_dataset_danmu,
    eval_dataset=small_eval_dataset_danmu,
    compute_metrics=compute_metrics,
    args=training_args
)


In [60]:
evaluator.evaluate()


***** Running Evaluation *****
  Num examples = 200
  Batch size = 4


KeyError: 0

In [115]:
# best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 4


{'eval_loss': 0.4616807699203491,
 'eval_accuracy': 0.94,
 'eval_runtime': 12.9227,
 'eval_samples_per_second': 15.477,
 'eval_steps_per_second': 3.869}

In [ ]:
# for n, v in best_run.hyperparameters.items():
#     setattr(trainer.args, n, v)
#
# trainer.train()

Try for other models
Is it possible to label the text with votes from different models?
Or, only judge whether the emotion is extreme on average
Or, predict hotness with the # of extreme emotion

Or, try to label the dan mu with emotion discovering?

In [ ]:
def compute_metrics_hotness(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    score = np.max(predictions)
    return metric.compute(predictions=predictions, references=labels)

In [54]:
# for each danmu, compute the its score value, marking positive/negative
# for each video, create hotness based on the number of danmu
small_eval_dataset_danmu[0:10].reset_index(drop=True)
small_eval_dataset_danmu = small_eval_dataset_danmu.rename(columns={"attention_masks": "attention_mask"})
small_train_dataset_danmu = small_train_dataset_danmu.rename(columns={"attention_masks": "attention_mask"})

In [ ]:
def predict_danmu(danmu_list):
    score_of_danmu = []
